In [46]:
import math
import numpy as np
import pandas as pd
import os
import tempfile

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from os import listdir
from os.path import isfile, join
path_train = 'ignored_assets/public_data/training_data/order_data'
trainfiles = [f for f in listdir(path_train) if isfile(join(path_train, f)) and f != '.DS_Store']

path_test1 = 'ignored_assets/public_data/test_set_1/order_data'
testfiles1 = [f for f in listdir(path_test1) if isfile(join(path_test1, f)) and f != '.DS_Store']

path_test2 = 'ignored_assets/public_data/test_set_2/order_data'
testfiles2 = [f for f in listdir(path_test2) if isfile(join(path_test2, f)) and f != '.DS_Store']


In [45]:
orderdf = []
for f in trainfiles:
    orderdf.append(pd.read_csv(join(path_train, f), delimiter='\t', header = None))
    
for f in testfiles1:
    orderdf.append(pd.read_csv(join(path_test1, f), delimiter='\t', header = None))
    
for f in testfiles2:
    orderdf.append(pd.read_csv(join(path_test2, f), delimiter='\t', header = None))




In [46]:
orderdf = pd.concat(orderdf)

In [5]:
#Clean up
orderdf[6] = pd.to_datetime(orderdf[6])
orderdf.columns = ['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash', 'price', 'timestamp']


In [6]:
# Create map of driver IDs
driver_ids=orderdf['driver_id'].unique()

In [57]:
#Index stuff
orderdf_f=orderdf[~pd.isnull(orderdf['driver_id'])]
myindex=pd.MultiIndex.from_arrays([orderdf_f['driver_id'],orderdf_f['timestamp']],names=('driver_id','timestamp'))

orderdf2=orderdf_f.set_index(myindex)
orderdf_s=orderdf2.sort_index()

%time orderdf_s.loc[driver_ids[0]]

CPU times: user 79 ms, sys: 23.7 ms, total: 103 ms
Wall time: 103 ms


,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,timestamp
timestamp,,,,,,,
2016-01-01 09:22:58,236c00d175a2295ade03ab6ea4b9689a,dd65fa250fca2833a3a8c16d2cf0457c,ce27cf85aaa48764f650414d71f3674e,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,6.0,2016-01-01 09:22:58
2016-01-01 09:35:26,a741062b69f6f1828d69a3f771aab6d5,dd65fa250fca2833a3a8c16d2cf0457c,1dc3e244fdef29a3299722f9aa26720f,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,6.0,2016-01-01 09:35:26
2016-01-01 09:43:39,c88282a71e988c5fb7eb11e33d5ca512,dd65fa250fca2833a3a8c16d2cf0457c,adbd59353c4651fc77405876d9e92638,62afaf3288e236b389af9cfdc5206415,d4ec2125aff74eded207d2d915ef682f,11.0,2016-01-01 09:43:39
2016-01-01 10:02:40,8fd177882b51161c854dbd1745a54c0c,dd65fa250fca2833a3a8c16d2cf0457c,2cf11c2f409e7c4503e0b47c6f7d2ba9,d4ec2125aff74eded207d2d915ef682f,4f4041f7db0c7f69892d9b74c1a7efa1,41.0,2016-01-01 10:02:40
2016-01-01 10:04:50,a174cc46d34b2b73bbb9b94a6d022b80,dd65fa250fca2833a3a8c16d2cf0457c,5d8886092bbb717e0ccf37cb2d7a9661,d4ec2125aff74eded207d2d915ef682f,d4ec2125aff74eded207d2d915ef682f,9.7,2016-01-01 10:04:50
2016-01-01 10:28:18,80559050428f704e024376a9d7306db9,dd65fa250fca2833a3a8c16d2cf0457c,4c5946a01d3764886040ba18fac717d6,d4ec2125aff74eded207d2d915ef682f,929ec6c160e6f52c20a4217c7978f681,11.0,2016-01-01 10:28:18
2016-01-01 10:30:13,5e03c747494657155b21320fa49144a9,dd65fa250fca2833a3a8c16d2cf0457c,da8ccb4ff0f510e46bb4643b8a785242,d4ec2125aff74eded207d2d915ef682f,929ec6c160e6f52c20a4217c7978f681,26.0,2016-01-01 10:30:13
2016-01-01 10:31:14,ea13ec3fe3b639eaefc36e4b54ebfd8e,dd65fa250fca2833a3a8c16d2cf0457c,729164967a31e6befeecc44548220cab,d4ec2125aff74eded207d2d915ef682f,82a0ceee5481d552b2bab973b5ed85cb,22.0,2016-01-01 10:31:14
2016-01-01 13:37:23,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23


In [50]:
#Index empty trips
orderdf_s['next_timestamp']=orderdf_s['timestamp'].shift(-1)
orderdf_s['next_start_district_hash']=orderdf_s['start_district_hash'].shift(-1)
orderdf_s['next_driver_id']=orderdf_s['driver_id'].shift(-1)

break_threshold=pd.Timedelta('1 hour')

orderdf_s['break']=(orderdf_s['next_timestamp']-orderdf_s['timestamp'])>break_threshold
orderdf_s['emptytrip']=(orderdf_s['dest_district_hash']!=orderdf_s['next_start_district_hash'])
orderdf_s['last_ride']=(orderdf_s['next_driver_id']!=orderdf_s['driver_id'])


In [53]:
#Compute number of rebalancing trips
num_reb_trips=sum(orderdf_s['emptytrip'] & ~orderdf_s['break'] & ~orderdf_s['last_ride'])
frac_reb_trips=float(num_reb_trips)/float(len(orderdf_s))
print 'Rebalancing trips: ', frac_reb_trips, 'of total'

Rebalancing trips:  0.278313998535 of total


In [79]:
#Output actual reb trips

reb_trips=orderdf_s[orderdf_s['emptytrip'] & ~orderdf_s['break'] & ~orderdf_s['last_ride']].copy()

In [83]:
#Group rebalancing trips
reb_trips.set_index('order_id')
reb_trips['quarter_hour'] = reb_trips['timestamp'].dt.round('15min')  
group_reb_trips=reb_trips.groupby(['quarter_hour','start_district_hash','dest_district_hash']).count()


In [84]:
group_reb_trips.head()

order_id  \
quarter_hour        start_district_hash              dest_district_hash                           
2016-01-01 00:00:00 1afd7afbc81ecc1b13886a569d869e8a 1afd7afbc81ecc1b13886a569d869e8a         6   
                                                     2407d482f0ffa22a947068f2551fe62c         3   
                                                     2920ece99323b4c111d6f9affc7ea034         3   
                                                     4396399c0f387a843c4de6052b51a69c         3   
                                                     4f8d81b5c31af5d1ba579a65ddc8a5cb         1   
                                                     62afaf3288e236b389af9cfdc5206415         1   
                                                     91690261186ae5bee8f83808ea1e4a01         2   
                                                     9868717243779473e6699d8a7bfce66f         1   
                                                     b702e920dcd2765e624dc1ce3a770512         1   
                    1cbfbdd079ef93e74405c53fcfff8567 1cbfbdd079ef93e74405c53fcfff8567         1   
                                                     307afa4120c590b3a46cf4ff5415608a         1   
                    1ecbb52d73c522f184a6fc53128b1ea1 5826ea92c0f1ca6fcfb12d90ec53f973         1   
                                                     7ffae45dfced1a5c82d1d6b124bfc666         1   
                    2301bc920194c95cf0c7486e5675243c 52a4e8aaa12f70020e889aed8fd5ddbc         1   
                    2350be163432e42270d2670cb3c02f80 82a0ceee5481d552b2bab973b5ed85cb         1   
                    2407d482f0ffa22a947068f2551fe62c 1afd7afbc81ecc1b13886a569d869e8a         3   
                                                     2407d482f0ffa22a947068f2551fe62c         3   
                                                     2920ece99323b4c111d6f9affc7ea034         4   
                                                     4396399c0f387a843c4de6052b51a69c         1   
                                                     62afaf3288e236b389af9cfdc5206415         2   
                                                     91690261186ae5bee8f83808ea1e4a01         5   
                                                     a5609739c6b5c2719a3752327c5e33a7         2   
                                                     d1ab2cc538d518758a1a82b1787592d4         1   
                                                     d4ec2125aff74eded207d2d915ef682f         2   
                                                     dd8d3b9665536d6e05b29c2648c0e69a         1   
                    2920ece99323b4c111d6f9affc7ea034 1afd7afbc81ecc1b13886a569d869e8a         2   
                                                     2407d482f0ffa22a947068f2551fe62c         3   
                                                     2498d22093591889d753dc151de1add9         1   
                                                     2920ece99323b4c111d6f9affc7ea034         1   
                                                     4725c39a5e5f4c188d382da3910b3f3f         1   
...                                                                                         ...   
2016-01-31 23:15:00 d4ec2125aff74eded207d2d915ef682f 82cc4851f9e4faa4e54309f8bb73fd7c         1   
                                                     929ec6c160e6f52c20a4217c7978f681         1   
                                                     ca064c2682ca48c6a21de012e87c0df5         1   
                                                     d4ec2125aff74eded207d2d915ef682f         3   
                                                     dd8d3b9665536d6e05b29c2648c0e69a         1   
                    d524868ce69cb9db10fc5af177fb9423 d524868ce69cb9db10fc5af177fb9423         1   
                    f2c8c4bb99e6377d21de71275afd6cd2 6986290e30a789dfd12f6d456f3c67cc         1   
                                                     ba32abfc048219e933bee869741da911         3   
                    f47f3524

In [85]:
#TODO: compare trips with outstanding pax requests

In [ ]:
#Find unfulfilled pax requests
orderdf_uf=orderdf[pd.isnull(orderdf['driver_id'])].copy()
orderdf_uf['quarter_hour'] = orderdf_uf['timestamp'].dt.round('15min')  
group_order_uf=orderdf_uf.groupby(['quarter_hour','start_district_hash']).count()
group_order_uf.head()

# Experiments of the failed kind

In [ ]:
def count_empty_trips(driver_hash):
    #Copy out driver info
    driver_orders=orderdf2.loc[driver_hash]
    driver_orders=driver_orders.sort_values('timestamp')
    
    #Auxiliary stuff
    driver_orders['next_timestamp']=driver_orders['timestamp'].shift(-1)
    driver_orders['next_start_district_hash']=driver_orders['start_district_hash'].shift(-1)
    break_threshold=pd.Timedelta('1 hour')
    
    #Did the driver go on break after the ride? Did the driver rebalance?
    driver_orders['break']=(driver_orders['next_timestamp']-driver_orders['timestamp'])>break_threshold
    driver_orders['emptytrip']=(driver_orders['dest_district_hash']!=driver_orders['next_start_district_hash'])
    
    if len(driver_orders)>0:
        return float(sum(driver_orders['emptytrip'] & ~driver_orders['break']))/float(len(driver_orders))
    else:
        return np.NaN

In [ ]:
driver_pd=pd.DataFrame(driver_ids[~pd.isnull(driver_ids)],columns=['driver_id'])

In [ ]:
driver_pd['frac_reb']=driver_pd.apply(lambda x: count_empty_trips(x['driver_id']),axis=1)

In [ ]:
orderdf2=orderdf.sort_values('driver_id')

myindex=pd.MultiIndex.from_tuples(list(zip(orderdf['driver_id'],orderdf['order_id'])))

orderdf3=orderdf2.set_index('driver_id')


#driver_orders=orderdf[orderdf['driver_id']==driver_hash].copy()

In [ ]:
driver_ids[pd.isnull(driver_ids)]

In [ ]:
list(zip(orderdf['driver_id'],orderdf['order_id']))

In [ ]:
count_empty_trips(driver_ids[0])

In [ ]:
#Create a timeline of rebalancing trips.
# For each, record two timestamps (last pickup, next pickup), start location, and end location.
# Sketch: for each driver, return a list of reb trips by masking. Then append.
# Can we do this in a non-sequential way?

In [ ]:
def find_empty_trips(driver_hash):
    #Copy out driver info
    driver_orders=orderdf2.loc[driver_hash]
    driver_orders=driver_orders.sort_values('timestamp')
    
    #Auxiliary stuff
    driver_orders['next_timestamp']=driver_orders['timestamp'].shift(-1)
    driver_orders['next_start_district_hash']=driver_orders['start_district_hash'].shift(-1)
    break_threshold=pd.Timedelta('1 hour')
    
    #Did the driver go on break after the ride? Did the driver rebalance?
    driver_orders['break']=(driver_orders['next_timestamp']-driver_orders['timestamp'])>break_threshold
    driver_orders['emptytrip']=(driver_orders['dest_district_hash']!=driver_orders['next_start_district_hash'])
    
    return driver_orders[driver_orders['emptytrip'] & ~driver_orders['break']]

In [ ]:
empty_rides_list=driver_pd.apply(lambda x: find_empty_trips(x['driver_id']),axis=1)